# Deep Gaussian Process

### Author: Mark Pullin

This notebooks demonstrates the use of deep Gaussian processes in MXFusion for regression. The model is based on the [Doubly Stochastic Variational Inference for Deep Gaussian Processes (Hugh Salimbeni, Marc Deisenroth)](https://arxiv.org/abs/1705.08933) paper.

This notebook fits the model to the ["kin8nm" dataset](https://www.openml.org/d/189).

In [1]:
import os
os.environ['MXNET_ENGINE_TYPE'] = 'NaiveEngine'
FIG_SIZE = (12, 8)

import mxfusion
from mxfusion.common import config
import mxnet as mx
dtype = 'float64'
config.DEFAULT_DTYPE = dtype

context = mx.gpu()
config.MXNET_DEFAULT_DEVICE = context

# Get training data

The training data will be downloaded from UCI repo, then the data is split into training and test set. 10% of the data is reserved for testing. The features and targets are then normalized by the mean and standard deviation of the training data.

In [2]:
import pandas as pd
data_path = 'https://www.openml.org/data/get_csv/3626/dataset_2175_kin8nm.arff'

df = pd.read_csv(data_path)

X = df.values[:, :-1]
Y = df.values[:, [-1]]

In [3]:
# Do train/test split and normalize data set with mean and standard deviation of training data
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.1)

x_std = x_train.std(axis=0)
x_mean = x_train.mean(axis=0)
y_std = y_train.std(axis=0)
y_mean = y_train.mean(axis=0)

x_train = (x_train - x_mean)/x_std
x_test = (x_test - x_mean)/x_std

y_train = (y_train - y_mean)/y_std
y_test = (y_test - y_mean)/y_std

# Make DGP model

This DGP has 3 layers, with hidden layers having dimensionality of 30. Each layer has 100 inducing points.

In [4]:
import mxnet as mx
import numpy as np
import mxfusion.components.distributions.gp.kernels
from mxfusion.components.variables import PositiveTransformation
from mxfusion.modules.gp_modules.deep_gp_regression import DeepGPRegression
from mxfusion.components.distributions.gp.kernels import RBF, White
from scipy.cluster.vq import kmeans2
from functools import partial

D = x_train.shape[1]

def make_mean_and_Z(n_dimensions, X, Z):
    """
    Function to create mean functions and inducing point locations for layers
    
    :param n_dimensions: List of length n_layers + 1 containing input/output dimensionality for each layer
    :param X: training data
    :param Z: inducing points at first layer
    """
    dtype = 'float64'
    
    def mx_dot(x, y):
        return mx.nd.dot(y, x)
    
    n_layers = len(n_dimensions) - 1
    
    X_running = mx.nd.array(X, dtype=dtype, ctx=mx.cpu())
    Z_running = mx.nd.array(Z, dtype=dtype, ctx=mx.cpu())
    
    mean_fcns = []
    inducing_points = []
    for i in range(n_layers):
        inducing_points.append(Z_running)
        input_dim = n_dimensions[i]
        output_dim = n_dimensions[i+1]
        
        print(input_dim, output_dim)
        if input_dim == output_dim:
            mean_mat = mx.nd.eye(input_dim, dtype=dtype, ctx=context)
        elif input_dim > output_dim:
            _, _, v = np.linalg.svd(X_running.asnumpy(), full_matrices=False)
            mean_mat = mx.nd.array(v, dtype=dtype, ctx=context)[:output_dim, :].T
            print(mean_mat.shape)
        else:
            # output_dim > input_dim
            padding_array = mx.nd.zeros((input_dim, output_dim - input_dim), dtype=dtype, ctx=context)
            mean_mat = mx.nd.concatenate([mx.nd.eye(input_dim, ctx=context, dtype=dtype), padding_array], axis=1)
        X_running = mx.nd.dot(X_running, mean_mat.as_in_context(mx.cpu()))
        Z_running = mx.nd.dot(Z_running, mean_mat.as_in_context(mx.cpu()))
        
        mean_fcns.append(partial(mx_dot, mean_mat))
        
    return mean_fcns, inducing_points

def make_dgp_model(n_dimensions, n_inducing, Z, mean_functions):
    """
    Makes a DGP model with specified latent dimensions with RBF kernels at each layer
    """
    layer_input_dimensions = n_dimensions[:-1]
    layer_output_dimensions = n_dimensions[1:]
    
    m = mxfusion.Model()
    m.N = mxfusion.Variable()
    
    n_layers = len(layer_input_dimensions)
    m.X = mxfusion.Variable(shape=(m.N, n_dimensions[0]))
    kernels = []
    for i in range(n_layers):
        M = n_inducing[i]
        kernels.append(RBF(layer_input_dimensions[i], lengthscale=2., variance=2., ctx=context, ARD=True) 
                      + White(layer_input_dimensions[i], variance=1e-5))
        z_locations = np.linspace(-1, 1, M)[:, None]
        setattr(m, 'Z_' + str(i), mxfusion.Variable(shape=(M, layer_input_dimensions[i]), 
                                                    initial_value=mx.nd.array(Z[i])))
        
    m.noise_var = mxfusion.Variable(transformation=PositiveTransformation())
    inducing_variables = [getattr(m, 'Z_' + str(i)) for i in range(n_layers)]
    m.Y = DeepGPRegression.define_variable(m.X, kernels, m.noise_var, shape=(m.N, n_dimensions[-1]), 
                                           inducing_inputs=inducing_variables, n_samples=5, 
                                           mean_functions=mean_functions)
    
    return m

Z_100 = kmeans2(x_train, 100, minit='points')[0]

M = 10

n_layers = 3
n_inducing = [100] * n_layers
n_dims_hidden_layers = min(D, 30)
n_dimensions = [D] + [n_dims_hidden_layers] * (n_layers - 1) + [1]
mean_fcns, inducing_locations = make_mean_and_Z(n_dimensions, x_train, Z_100)
m = make_dgp_model(n_dimensions, n_inducing, inducing_locations, mean_fcns)
n_layers = m.Y.factor.n_layers

8 8
8 8
8 1
(8, 1)


# Create inference object

In [5]:
from mxfusion.inference import GradBasedInference, MAP, MinibatchInferenceLoop

batch_size = min(10000, x_train.shape[0]/2)
infr = GradBasedInference(inference_algorithm=MAP(model=m, observed=[m.X, m.Y]), 
                          grad_loop=MinibatchInferenceLoop(batch_size), dtype=dtype)

In [6]:
infr.initialize(X=x_train.shape, Y=y_train.shape)

/home/ubuntu/git/MXFusion/mxfusion/inference/inference_parameters.py:71: UserWarning: InferenceParameters has already been initialized.  The existing one will be overwritten.
  warnings.warn("InferenceParameters has already been initialized.  The existing one will be overwritten.")


# Initialize parameter values

The variational distribution at each layer is initialized to have zero mean, identity covariance matrix scaled by 1e-5. The noise variance is initialized to 0.01.

In [7]:
from functools import partial
def initialise_model(m, infr):
    """
    Initializes mean of variational distribution at inducing points to be identity function
    """
    dtype = 'float64'
    infr.initialize(X=mx.nd.array(X, dtype=dtype, ctx=context), 
                    Y=mx.nd.array(Y, dtype=dtype, ctx=context))
    
    def mx_dot(x, y):
        return mx.nd.dot(y, x)
    
    mean_fcns = []
    for i in range(n_layers):
        mean = getattr(m.Y.factor._extra_graphs[0], 'qU_mean_' + str(i))
        cov_w = getattr(m.Y.factor._extra_graphs[0], 'qU_cov_W_' + str(i))
        cov_diag = getattr(m.Y.factor._extra_graphs[0], 'qU_cov_diag_' + str(i))
        infr.params[cov_w] = mx.nd.zeros_like(infr.params[cov_w])
        infr.params[cov_diag] = mx.nd.ones_like(infr.params[cov_diag]) * 1e-5
        infr.params[mean] = mx.nd.zeros_like(infr.params[mean])
    infr.params[m.noise_var] = mx.nd.array([0.01], ctx=context)
initialise_model(m, infr)
m.Y.factor.dgp_log_pdf.jitter = 1e-6

/home/ubuntu/git/MXFusion/mxfusion/inference/inference.py:163: UserWarning: Trying to initialize the inference twice, skipping.
  warnings.warn("Trying to initialize the inference twice, skipping.")


# Train model

In [8]:
infr.run(X=mx.nd.array(x_train, dtype='float64', ctx=context), 
         Y=mx.nd.array(y_train, dtype='float64', ctx=context), max_iter=2000, learning_rate=0.01)


/home/ubuntu/git/MXFusion/mxfusion/inference/inference.py:163: UserWarning: Trying to initialize the inference twice, skipping.
  warnings.warn("Trying to initialize the inference twice, skipping.")


# Predict from model

In [9]:
m.Y.factor.dgp_predict.noise_free = False
from mxfusion.inference import ModulePredictionAlgorithm, TransferInference
infr_pred = TransferInference(ModulePredictionAlgorithm(model=m, observed=[m.X], target_variables=[m.Y]), 
                              infr_params=infr.params)

res = infr_pred.run(X=mx.nd.array(x_test, dtype='float64', ctx=context))

mean_samples, var_samples = res[0][0].asnumpy(), res[0][1].asnumpy()

mean = mean_samples.mean(axis=0)
var = (var_samples + mean_samples**2).mean(axis=0) - mean**2

/home/ubuntu/git/MXFusion/mxfusion/inference/inference_parameters.py:71: UserWarning: InferenceParameters has already been initialized.  The existing one will be overwritten.
  warnings.warn("InferenceParameters has already been initialized.  The existing one will be overwritten.")


In [10]:
rmse = np.sqrt(np.mean(np.square((mean - y_test)))) * y_std

In [11]:
import scipy.stats
test_log_likelihood = scipy.stats.norm.logpdf(y_test * y_std, mean * y_std, np.sqrt(var) * y_std).mean()

In [12]:
print('RMSE; {:.4f}'.format(rmse[0]))
print('Log likelihood: {:.4f}'.format(test_log_likelihood))

RMSE; 0.0681
Log likelihood: 1.2289


# Compare against single layer SVGP

In [13]:
svgp_model = mxfusion.Model()
svgp_model.N = mxfusion.Variable()
svgp_model.X = mxfusion.Variable(shape=(svgp_model.N, D))
svgp_model.Z= mxfusion.Variable(initial_value=mx.nd.array(Z_100, dtype=dtype, ctx=context), shape=(100, D))
svgp_model.noise_var = mxfusion.Variable(transformation=PositiveTransformation(), initial_value=0.01)
kernel = RBF(D, ARD=True)
svgp_model.Y = mxfusion.modules.gp_modules.SVGPRegression.define_variable(svgp_model.X, kernel, svgp_model.noise_var, 
                                                                          shape=(svgp_model.N, 1), 
                                                                          inducing_inputs=svgp_model.Z)

In [14]:
svgp_infr = GradBasedInference(inference_algorithm=MAP(model=svgp_model, observed=[svgp_model.X, svgp_model.Y]), 
                          grad_loop=MinibatchInferenceLoop(batch_size), dtype=dtype)
svgp_infr.initialize(X=x_train.shape, Y=y_train.shape)

/home/ubuntu/git/MXFusion/mxfusion/inference/inference_parameters.py:71: UserWarning: InferenceParameters has already been initialized.  The existing one will be overwritten.
  warnings.warn("InferenceParameters has already been initialized.  The existing one will be overwritten.")


In [15]:
svgp_infr.run(X=mx.nd.array(x_train, dtype='float64', ctx=context), 
              Y=mx.nd.array(y_train, dtype='float64', ctx=context), max_iter=2000, learning_rate=0.01)

/home/ubuntu/git/MXFusion/mxfusion/inference/inference.py:163: UserWarning: Trying to initialize the inference twice, skipping.
  warnings.warn("Trying to initialize the inference twice, skipping.")


In [16]:
svgp_infr_pred = TransferInference(ModulePredictionAlgorithm(model=svgp_model, observed=[svgp_model.X], 
                                                             target_variables=[svgp_model.Y]), 
                                   infr_params=svgp_infr.params)

svgp_model.Y.factor.svgp_predict.noise_free = False
res = svgp_infr_pred.run(X=mx.nd.array(x_test, dtype='float64', ctx=context))

mean, var = res[0][0].asnumpy(), res[0][1].asnumpy()

/home/ubuntu/git/MXFusion/mxfusion/inference/inference_parameters.py:71: UserWarning: InferenceParameters has already been initialized.  The existing one will be overwritten.
  warnings.warn("InferenceParameters has already been initialized.  The existing one will be overwritten.")


In [17]:
rmse = np.sqrt(np.mean(np.square((mean - y_test)))) * y_std
test_log_likelihood = scipy.stats.norm.logpdf(y_test * y_std, mean * y_std, np.sqrt(var) * y_std).mean()

print('RMSE; {:.4f}'.format(rmse[0]))
print('Log likelihood: {:.4f}'.format(test_log_likelihood))

RMSE; 0.0876
Log likelihood: 0.9679


We can see the DGP outperforms the single layer GP for this dataset.